# **PART 1 : Data Cleaning**

In [2]:
import pandas as pd
from itertools import combinations

In [3]:
def charger_dataset(file_name):
    """Charge un fichier CSV et retourne le df."""
    with open(file_name, 'r') as f:
        df = pd.read_csv(f, delimiter=',')
        return df
    
print(charger_dataset('Dataset-Exos2.csv'))

    Watcher  videoCategoryId     videoCategoryLabel definition
0     Billy             29.0  Nonprofits & Activism         hd
1     Leila             22.0         People & Blogs         sd
2     Billy             22.0         People & Blogs         sd
3      Mark             24.0          Entertainment         hd
4     Billy             24.0          Entertainment         hd
..      ...              ...                    ...        ...
994     NaN              NaN                    NaN        NaN
995     NaN              NaN                    NaN        NaN
996     NaN              NaN                    NaN        NaN
997     NaN              NaN                    NaN        NaN
998     NaN              NaN                    NaN        NaN

[999 rows x 4 columns]


In [4]:
dataset2 = charger_dataset('Dataset-Exos2.csv')
# Nombre de transactions (nombre de lignes)
nombre_de_transactions = len(dataset2)
# Nombre d'items (nombre de valeurs distinctes dans la colonne "videoCategoryId")
nombre_d_items = dataset2['videoCategoryId'].nunique()
print("Nombre de transactions : ", nombre_de_transactions)
print("Nombre d'items : ", nombre_d_items)

Nombre de transactions :  999
Nombre d'items :  13


In [5]:
def pretraitement(dataset):
    # Supprimer les lignes contenant des valeurs manquantes (NaN)
    dataset.dropna(inplace=True) 
    return dataset

dataset2 = pretraitement(dataset2)
len(dataset2)
dataset2.head(10)

,Watcher,videoCategoryId,videoCategoryLabel,definition
0,Billy,29.0,Nonprofits & Activism,hd
1,Leila,22.0,People & Blogs,sd
2,Billy,22.0,People & Blogs,sd
3,Mark,24.0,Entertainment,hd
4,Billy,24.0,Entertainment,hd
5,Jane,24.0,Entertainment,hd
6,Babs,22.0,People & Blogs,hd
7,Jeff,25.0,News & Politics,hd
9,Leila,28.0,Science & Technology,hd
10,Jane,27.0,Education,hd


In [6]:
dataset2_bis = dataset2.groupby('Watcher')['videoCategoryId'].apply(set).reset_index()
#renaming the columns
dataset2_bis.rename(columns={'Watcher':'Transaction','videoCategoryId':'Items'},inplace=True)
dataset2_bis.head(10)

,Transaction,Items
0,Adam,{22.0}
1,Alex,{22.0}
2,Amy,{22.0}
3,Babs,{22.0}
4,Ben,{22.0}
5,Billy,"{22.0, 24.0, 26.0, 28.0, 29.0}"
6,Bob,"{27.0, 28.0, 22.0}"
7,Brad,"{10.0, 28.0, 29.0, 22.0}"
8,Chandler,{28.0}
9,Clark,"{28.0, 22.0}"


In [7]:
n = len(dataset2_bis)
print('le nombre de transaction est :', n)
len_items = dataset2_bis['Items'].explode().unique()
print("le nombre d'item :", len(len_items))
len_items

le nombre de transaction est : 39
le nombre d'item : 12


array([22.0, 24.0, 26.0, 28.0, 29.0, 27.0, 10.0, 20.0, 25.0, 1.0, 23.0,
       17.0], dtype=object)

# **PART 2 : Apriori**

In [8]:
def generate_candidates(k):
    """
    Génère les k-itemsets candidats Ck à partir du dataset.
    
    :param dataset: Le dataset au format transactionnel
    :param k: La taille des itemsets candidats à générer (k)
    
    :return: Liste des k-itemsets candidats Ck
    """
    ck = []
    # Créer un ensemble de tous les items uniques présents dans le dataset 
    items = set()
    for i in dataset2_bis['Items']:
        for j in i:
            items.add(j)
    items = list(items)
    # Générer les combinaisons d'items de taille k
    comb = combinations(items, k)
    # Ajouter les combinaisons d'items de taille k à la liste ck
    for i in list(comb):
        ck.append(i)
    return ck

Support : Il s'agit de la fréquence avec laquelle une combinaison d'articles (ou items) apparaît dans l'ensemble des transactions. Le support est utilisé pour identifier les éléments fréquents.

In [9]:
def calculate_support(dataset, candidates):
    """
    Calcule le support de chaque k-itemset candidat dans le dataset.
    
    :param dataset: Le dataset au format transactionnel
    :param candidates: Liste des k-itemsets candidats Ck
    :return: Un dictionnaire contenant les supports de chaque k-itemset
    """
    support_counts = {}  # Dictionnaire pour stocker le support de chaque k-itemset
    
    for candidate in candidates:
        candidate_set = set(candidate)
        for index, row in dataset.iterrows():
            if candidate_set.issubset(row['Items']):
                if tuple(candidate) in support_counts:
                    support_counts[tuple(candidate)] += 1
                else:
                    support_counts[tuple(candidate)] = 1
    
    return support_counts

In [10]:
def generate_frequent_itemsets(support_counts, min_support):
    """
    Génère les k-itemsets fréquents Lk à partir des supports des k-itemsets candidats.
    
    :param support_counts: Dictionnaire contenant les supports de chaque k-itemset
    :param min_support: Le support minimum pour qu'un k-itemset soit considéré comme fréquent
    :return: Liste des k-itemsets fréquents Lk
    """
    Lk = []  # Liste des k-itemsets fréquents
    
    for candidate, support in support_counts.items():
        if support >= min_support:
            Lk.append(candidate)
    
    return Lk

In [14]:
# testing the functions 
C2 = generate_candidates(2)
print("C2 :", C2)
support_counts = calculate_support(dataset2_bis, C2)
print("support_counts :", support_counts)
L2 = generate_frequent_itemsets(support_counts, 5)
print("L2 :", L2)

C2 : [(1.0, 10.0), (1.0, 17.0), (1.0, 20.0), (1.0, 22.0), (1.0, 23.0), (1.0, 24.0), (1.0, 25.0), (1.0, 26.0), (1.0, 27.0), (1.0, 28.0), (1.0, 29.0), (10.0, 17.0), (10.0, 20.0), (10.0, 22.0), (10.0, 23.0), (10.0, 24.0), (10.0, 25.0), (10.0, 26.0), (10.0, 27.0), (10.0, 28.0), (10.0, 29.0), (17.0, 20.0), (17.0, 22.0), (17.0, 23.0), (17.0, 24.0), (17.0, 25.0), (17.0, 26.0), (17.0, 27.0), (17.0, 28.0), (17.0, 29.0), (20.0, 22.0), (20.0, 23.0), (20.0, 24.0), (20.0, 25.0), (20.0, 26.0), (20.0, 27.0), (20.0, 28.0), (20.0, 29.0), (22.0, 23.0), (22.0, 24.0), (22.0, 25.0), (22.0, 26.0), (22.0, 27.0), (22.0, 28.0), (22.0, 29.0), (23.0, 24.0), (23.0, 25.0), (23.0, 26.0), (23.0, 27.0), (23.0, 28.0), (23.0, 29.0), (24.0, 25.0), (24.0, 26.0), (24.0, 27.0), (24.0, 28.0), (24.0, 29.0), (25.0, 26.0), (25.0, 27.0), (25.0, 28.0), (25.0, 29.0), (26.0, 27.0), (26.0, 28.0), (26.0, 29.0), (27.0, 28.0), (27.0, 29.0), (28.0, 29.0)]
support_counts : {(1.0, 22.0): 1, (1.0, 23.0): 1, (1.0, 28.0): 1, (1.0, 29.0): 1,

# **Partie 03 : Règles d'association**
Les règles d'association sont utilisées dans l'exploration de données pour découvrir des relations fréquentes entre les variables d'un ensemble de données.

In [15]:
def generate_association_rules(Lk):
    association_rules = []
    for itemset in Lk:
        # convert type to set 
        itemset = set(itemset)
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    antecedent = set(antecedent)
                    consequent = itemset - antecedent
                    association_rules.append((antecedent, consequent))
    return association_rules

In [16]:
C3 = generate_candidates(3)
L3 = generate_frequent_itemsets(calculate_support(dataset2_bis, C3), 5)
association_rules = generate_association_rules(L3)

# create a dataframe to store the result
association_rules_df = pd.DataFrame(columns=['Antecedent', 'Consequent'])
association_rules_df['Antecedent'] = [antecedent for antecedent, consequent in association_rules]
association_rules_df['Consequent'] = [consequent for antecedent, consequent in association_rules]
association_rules_df.head(20)

,Antecedent,Consequent
0,{28.0},"{29.0, 22.0}"
1,{29.0},"{28.0, 22.0}"
2,{22.0},"{28.0, 29.0}"
3,"{28.0, 29.0}",{22.0}
4,"{28.0, 22.0}",{29.0}
5,"{29.0, 22.0}",{28.0}


Confiance : La confiance mesure la probabilité conditionnelle que, si un client achète un ensemble d'articles, il achètera également un autre article. 

Confiance (A ⇒ B) = P(B/A) = support(A ∪ B) / support(A)

In [13]:
def calculate_confidence(transaction_data, antecedent, consequent):
    # Nombre de transactions contenant l'antécédent et le conséquent
    support_A_and_B = sum(1 for transaction in transaction_data if antecedent.issubset(transaction) and consequent.issubset(transaction))
    
    # Nombre de transactions contenant l'antécédent
    support_A = sum(1 for transaction in transaction_data if antecedent.issubset(transaction))
    
    # Calcul de la confiance
    confidence = support_A_and_B / support_A if support_A > 0 else 0
    return confidence

# print the result
association_rules_df['Confidence'] = association_rules_df.apply(lambda x: calculate_confidence(dataset2_bis['Items'], x['Antecedent'], x['Consequent']), axis=1)
association_rules_df.head(20)

,Antecedent,Consequent,Confidence
0,{28.0},"{29.0, 22.0}",0.333333
1,{29.0},"{28.0, 22.0}",0.888889
2,{22.0},"{28.0, 29.0}",0.266667
3,"{28.0, 29.0}",{22.0},0.888889
4,"{28.0, 22.0}",{29.0},0.470588
5,"{29.0, 22.0}",{28.0},1.000000


In [20]:
def calculate_confidence(transaction_data, antecedent, consequent):
    # Nombre de transactions contenant l'antécédent et le conséquent
    support_A_and_B = sum(1 for transaction in transaction_data if antecedent.issubset(transaction) and consequent.issubset(transaction))
    
    # Nombre de transactions contenant l'antécédent
    support_A = sum(1 for transaction in transaction_data if antecedent.issubset(transaction))
    
    # Calcul de la confiance
    confidence = support_A_and_B / support_A if support_A > 0 else 0
    return support_A_and_B, support_A, confidence  # Retourner les valeurs dans un tuple

# Appliquer la fonction calculate_confidence et stocker les résultats dans des listes
results = association_rules_df.apply(lambda x: calculate_confidence(dataset2_bis['Items'], x['Antecedent'], x['Consequent']), axis=1)

# Diviser les valeurs du tuple dans différentes listes
support_A_and_B_values = [res[0] for res in results]
support_A_values = [res[1] for res in results]
confidence_values = [res[2] for res in results]

# Ajouter les nouvelles colonnes avec les valeurs calculées
association_rules_df['Support_A_and_B'] = support_A_and_B_values
association_rules_df['Support_A'] = support_A_values
association_rules_df['Confidence'] = confidence_values

# Afficher les premières lignes du DataFrame avec les nouvelles colonnes ajoutées
print(association_rules_df.head(20))


     Antecedent    Consequent  Support_A_and_B  Support_A  Confidence
0        {28.0}  {29.0, 22.0}                8         24    0.333333
1        {29.0}  {28.0, 22.0}                8          9    0.888889
2        {22.0}  {28.0, 29.0}                8         30    0.266667
3  {28.0, 29.0}        {22.0}                8          9    0.888889
4  {28.0, 22.0}        {29.0}                8         17    0.470588
5  {29.0, 22.0}        {28.0}                8          8    1.000000
